In [1]:
import warnings
import flwr as fl
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

import utils

if __name__ == "__main__":
    print('starting flow client')
    print('load Data')
    (X_train, y_train), (X_test, y_test) = utils.load_bankingdata()
    print(X_train.shape)
    # Split train set into 10 partitions and randomly use one for training.
    #partition_id = np.random.choice(10)
    #(X_train, y_train) = utils.partition(X_train, y_train, 10)[partition_id]
    print('data loaded')
    # Create LogisticRegression Model
    model = LogisticRegression(
        penalty="l2",
        max_iter=1,  # local epoch
        warm_start=True,  # prevent refreshing weights when fitting
    )
    print("Model created")
    print(X_train.shape[0])

    # Setting initial parameters, akin to model.compile for keras models
    utils.set_initial_params(model, 14, 2)

    # Define Flower client
    class FlowerClient(fl.client.NumPyClient):
        def get_parameters(self, config):  # type: ignore
            return utils.get_model_parameters(model)

        def fit(self, parameters, config):  # type: ignore
            utils.set_model_params(model, parameters)
            # Ignore convergence failure due to low local epochs
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model.fit(X_train, y_train)
            print(f"Training finished for round {config['server_round']}")
            return utils.get_model_parameters(model), len(X_train), {}

        def evaluate(self, parameters, config):  # type: ignore
            utils.set_model_params(model, parameters)
            loss = log_loss(y_test, model.predict_proba(X_test))
            accuracy = model.score(X_test, y_test)
            return loss, len(X_test), {"accuracy": accuracy}

    # Start Flower client
    fl.client.start_numpy_client(server_address="0.0.0.0:8080", client=FlowerClient())


starting flow client
load Data


INFO flwr 2023-09-16 17:55:08,649 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-09-16 17:55:08,651 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-09-16 17:55:08,652 | connection.py:39 | ChannelConnectivity.CONNECTING
DEBUG flwr 2023-09-16 17:55:08,654 | connection.py:39 | ChannelConnectivity.TRANSIENT_FAILURE


(173444, 21365)
data loaded
Model created
173444


DEBUG flwr 2023-09-16 17:55:08,858 | connection.py:113 | gRPC channel closed


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:0.0.0.0:8080: Failed to connect to remote host: Connection refused"
	debug_error_string = "UNKNOWN:failed to connect to all addresses; last error: UNKNOWN: ipv4:0.0.0.0:8080: Failed to connect to remote host: Connection refused {created_time:"2023-09-16T17:55:08.653948+02:00", grpc_status:14}"
>